In [ ]:
!pip install patchify

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import glob
from matplotlib import pyplot as plt

from patchify import patchify
import tifffile as tiff
from PIL import Image
import tensorflow as tf
from tensorflow import keras
# import segmentation_models as sm
from tensorflow.keras.metrics import MeanIoU
import random

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
temp_img = cv2.imread("/kaggle/input/potsdam/Potsdam/2_Ortho_RGB/2_Ortho_RGB/top_potsdam_2_10_RGB.tif") #3 channels / spectral bands

# fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
# fig.suptitle('Channel wise image of the Red, Green & Infrared spectral band')

plt.imshow(temp_img[:,:,2]) #View each channel...
# ax2.imshow(temp_img[:,:,1])
# ax3.imshow(temp_img[:,:,2])

temp_mask = cv2.imread("/kaggle/input/potsdam/Potsdam/5_Labels_all/top_potsdam_2_10_label.tif") #3 channels but all same.
temp_mask_gray = cv2.cvtColor(temp_mask, cv2.COLOR_BGR2GRAY)
# labels, count = np.unique(temp_mask_gray[:,:,0], return_counts=True) #Check for each channel. All chanels are identical


scaling_factor = 5 / 255.0

# Apply the scaling to the image
scaled_mask = temp_mask_gray * scaling_factor

# Convert to integer values
scaled_mask = scaled_mask.astype(np.uint8)


labels2, count2 = np.unique(temp_mask_gray, return_counts=True)
labels, count = np.unique(scaled_mask, return_counts=True)
print("Labels are: ", labels, " and the counts are: ", count)
print("Labels are: ", labels2, " and the counts are: ", count2)

In [ ]:
plt.imshow(scaled_mask)

In [ ]:
# import shutil
# shutil.rmtree("/kaggle/working/main_folder/sub_folder")    ## to delete the directory

In [ ]:
from pathlib import Path
Path('/kaggle/working/main_folder/512res_patches').mkdir(parents=True, exist_ok=True)
Path('/kaggle/working/main_folder/512res_patches/images').mkdir(parents=True, exist_ok=True)
Path('/kaggle/working/main_folder/512res_patches/masks').mkdir(parents=True, exist_ok=True)
Path('/kaggle/working/main_folder/data_for_training_and_testing').mkdir(parents=True, exist_ok=True)

In [ ]:
root_directory = '/kaggle/input/potsdam/Potsdam'
patch_size = 384
img_dir="/kaggle/input/potsdam/Potsdam/2_Ortho_RGB/2_Ortho_RGB"
print("1")

In [ ]:
for path, subdirs, files in os.walk(img_dir):
    #print(path)  
    dirname = path.split(os.path.sep)[-1]
    #print(dirname)
    images = os.listdir(path)  #List of all image names in this subdirectory
    #print(images)
    for i, image_name in enumerate(images):  
        if image_name.endswith(".tif"):
            #print(image_name)
            image = cv2.imread(path+"/"+image_name, 1)  #Read each image as BGR
            SIZE_X = (image.shape[1]//patch_size)*patch_size #Nearest size divisible by our patch size
            SIZE_Y = (image.shape[0]//patch_size)*patch_size #Nearest size divisible by our patch size
            image = Image.fromarray(image)
            image = image.crop((0 ,0, SIZE_X, SIZE_Y))  #Crop from top left corner
            #image = image.resize((SIZE_X, SIZE_Y))  #Try not to resize for semantic segmentation
            image = np.array(image)             
   
            #Extract patches from each image
            print("Now patchifying image:", path+"/"+image_name)
            patches_img = patchify(image, (384, 384, 3), step=512)  #Step=512 for 256 patches means no overlap
            new_name = image_name.replace('_RGB.tif', '')
            
            for i in range(patches_img.shape[0]):
                for j in range(patches_img.shape[1]):
                    
                    single_patch_img = patches_img[i,j,:,:]
                    #single_patch_img = (single_patch_img.astype('float32')) / 255. #We will preprocess using one of the backbones
                    single_patch_img = single_patch_img[0] #Drop the extra unecessary dimension that patchify adds.                               
                    
                    cv2.imwrite("/kaggle/working/main_folder/512res_patches/images/"+
                               new_name+"patch_" + str(i) + str(j) + ".tif", single_patch_img)
                    #image_dataset.append(single_patch_img)


In [ ]:
mask_dir="/kaggle/input/potsdam/Potsdam/5_Labels_all"
for path, subdirs, files in os.walk(mask_dir):
    #print(path)  
    dirname = path.split(os.path.sep)[-1]

    masks = os.listdir(path)  #List of all image names in this subdirectory
    for i, mask_name in enumerate(masks):  
        if mask_name.endswith(".tif"):           
            mask = cv2.imread(path+"/"+mask_name, 0)  #Read each image as Grey (or color but remember to map each color to an integer)
            SIZE_X = (mask.shape[1]//patch_size)*patch_size #Nearest size divisible by our patch size
            SIZE_Y = (mask.shape[0]//patch_size)*patch_size #Nearest size divisible by our patch size
            mask = Image.fromarray(mask)
            mask = mask.crop((0 ,0, SIZE_X, SIZE_Y))  #Crop from top left corner
            #mask = mask.resize((SIZE_X, SIZE_Y))  #Try not to resize for semantic segmentation
            mask = np.array(mask)             
   
            #Extract patches from each image
            print("Now patchifying mask:", path+"/"+mask_name)
            patches_mask = patchify(mask, (384, 384), step=512)  #Step=256 for 256 patches means no overlap
            new_name2 = mask_name.replace('_label.tif', '')
    
            for i in range(patches_mask.shape[0]):
                for j in range(patches_mask.shape[1]):
                    
                    single_patch_mask = patches_mask[i,j,:,:]
                    #single_patch_img = (single_patch_img.astype('float32')) / 255. #No need to scale masks, but you can do it if you want
                    #single_patch_mask = single_patch_mask[0] #Drop the extra unecessary dimension that patchify adds.                               
                    cv2.imwrite("/kaggle/working/main_folder/512res_patches/masks/"+
                               new_name2+"patch_"+str(i)+str(j)+".tif", single_patch_mask)

In [ ]:
train_img_dir = "/kaggle/working/main_folder/512res_patches/images/"
train_mask_dir = "/kaggle/working/main_folder/512res_patches/masks/"
img_list = os.listdir(train_img_dir)
msk_list = os.listdir(train_mask_dir)

In [ ]:
len(msk_list), len(img_list)

In [ ]:
num_images = len(os.listdir(train_img_dir))
img_num = random.randint(0, num_images-1)

img_for_plot = cv2.imread(train_img_dir+img_list[img_num], 1)
img_for_plot = cv2.cvtColor(img_for_plot, cv2.COLOR_BGR2RGB)

mask_for_plot =cv2.imread(train_mask_dir+msk_list[img_num], 0)

plt.figure(figsize=(12, 8))
plt.subplot(121)
plt.imshow(img_for_plot)
plt.title('Image')
plt.subplot(122)
plt.imshow(mask_for_plot, cmap='gray')
plt.title('Mask')
plt.show()

In [ ]:
from keras.utils import normalize
from keras.metrics import MeanIoU

In [ ]:
!pip install split-folders

In [ ]:
import splitfolders
input_folder = '/kaggle/working/main_folder/512res_patches'
output_folder = '/kaggle/working/main_folder/data_for_training_and_testing'
# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio(input_folder, output=output_folder, seed=32, ratio=(.6, .4), group_prefix=None) # default values

In [ ]:
train_img_dir = "/kaggle/working/main_folder/data_for_training_and_testing/train/images/"
train_mask_dir = "/kaggle/working/main_folder/data_for_training_and_testing/train/masks/"

img_list = os.listdir(train_img_dir)
msk_list = os.listdir(train_mask_dir)

num_images = len(os.listdir(train_img_dir))
img_num = random.randint(0, num_images-1)

img_for_plot = cv2.imread(train_img_dir+img_list[img_num], 1)
img_for_plot = cv2.cvtColor(img_for_plot, cv2.COLOR_BGR2RGB)

mask_for_plot =cv2.imread(train_mask_dir+msk_list[img_num], 0)

plt.figure(figsize=(12, 8))
plt.subplot(121)
plt.imshow(img_for_plot)
plt.title('Image')
plt.subplot(122)
plt.imshow(mask_for_plot, cmap='gray')
plt.title('Mask')
plt.show()

In [ ]:
# !pip install segmentation-models-pytorch

In [ ]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
# import segmentation_models_pytorch as smp
from tensorflow.keras.metrics import MeanIoU
import random
import tensorflow as tf
import keras 

from keras.utils import normalize
from keras.metrics import MeanIoU
import glob
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.dataset import Dataset
import albumentations as A

In [ ]:
train_img_dir = "/kaggle/working/main_folder/data_for_training_and_testing/train/images/"
train_mask_dir = "/kaggle/working/main_folder/data_for_training_and_testing/train/masks/"

img_list = os.listdir(train_img_dir)
msk_list = os.listdir(train_mask_dir)

num_images = len(os.listdir(train_img_dir))
img_num = random.randint(0, num_images-1)

img_for_plot = cv2.imread(train_img_dir+img_list[img_num], 1)
img_for_plot = cv2.cvtColor(img_for_plot, cv2.COLOR_BGR2RGB)

mask_for_plot =cv2.imread(train_mask_dir+msk_list[img_num], 0)

plt.figure(figsize=(12, 8))
plt.subplot(121)
plt.imshow(img_for_plot)
plt.title('Image')
plt.subplot(122)
plt.imshow(mask_for_plot, cmap='gray')
plt.title('Mask')
plt.show()

In [ ]:
!pip install segmentation-models 

In [ ]:
import glob
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt

!pip install -U -q segmentation-models
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

from tensorflow import keras
import segmentation_models as sm

In [ ]:
import keras 

from keras.utils import normalize
from keras.metrics import MeanIoU

In [ ]:
SIZE_X = 384 
SIZE_Y = 384
n_classes=6 #Number of classes for segmentation

In [ ]:
#Capture training image info as a list
train_images = []

for directory_path in glob.glob("/kaggle/working/main_folder/data_for_training_and_testing/val/images/"):
    for img_path in glob.glob(os.path.join(directory_path, "*.tif")):
        img = cv2.imread(img_path, 1)       
        #img = cv2.resize(img, (SIZE_Y, SIZE_X))
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        train_images.append(img)
       
#Convert list to array for machine learning processing        
train_images = np.array(train_images)

#Capture mask/label info as a list
train_masks = [] 
for directory_path in glob.glob("/kaggle/working/main_folder/data_for_training_and_testing/val/masks/"):
    for mask_path in glob.glob(os.path.join(directory_path, "*.tif")):
        mask = cv2.imread(mask_path, 0) 
        mask = mask*(5/255)
        mask = mask.astype(np.uint8)
        #mask = cv2.resize(mask, (SIZE_Y, SIZE_X), interpolation = cv2.INTER_NEAREST)  #Otherwise ground truth changes due to interpolation
        train_masks.append(mask)
        
#Convert list to array for machine learning processing          
train_masks = np.array(train_masks)

In [ ]:
train_images.shape, train_masks.shape 

In [ ]:
#Encode labels... but multi dim array so need to flatten, encode and reshape
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
n, h, w = train_masks.shape
train_masks_reshaped = train_masks.reshape(-1,1)
train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped)
train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)

np.unique(train_masks_encoded_original_shape)


In [ ]:
train_masks_input = np.expand_dims(train_masks_encoded_original_shape, axis=3)

#Create a subset of data for quick testing
#Picking 10% for testing and remaining for training
from sklearn.model_selection import train_test_split
X1, X_test, y1, y_test = train_test_split(train_images, train_masks_input, test_size = 0.10, random_state = 0)

#Further split training data t a smaller subset for quick testing of models
X_train, X_do_not_use, y_train, y_do_not_use = train_test_split(X1, y1, test_size = 0.1, random_state = 0)

print("Class values in the dataset are ... ", np.unique(y_train))  # 0 is the background/few unlabeled 


In [ ]:
from keras.utils import to_categorical
train_masks_cat = to_categorical(y_train, num_classes=n_classes)
y_train_cat = train_masks_cat.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2], n_classes))



test_masks_cat = to_categorical(y_test, num_classes=n_classes)
y_test_cat = test_masks_cat.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2], n_classes))

In [ ]:
X_train.shape, y_train_cat.shape

In [ ]:
n_classes=6
activation='softmax'

LR = 0.0001
optim = keras.optimizers.Adam(LR)

# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
# set class weights for dice_loss (car: 1.; pedestrian: 2.; background: 0.5;)
dice_loss = sm.losses.DiceLoss(class_weights=np.array([0.65528844, 3.56154108, 1.07120728, 0.73141398, 6.44243807,
       0.57563383])) 
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

# actulally total_loss can be imported directly from library, above example just show you how to manipulate with losses
# total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss 

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

In [ ]:
BACKBONE1 = 'resnet50'
preprocess_input1 = sm.get_preprocessing(BACKBONE1)

# preprocess input
X_train1 = preprocess_input1(X_train)
X_test1 = preprocess_input1(X_test)

In [ ]:
# define model
model1 = sm.Unet(BACKBONE1, encoder_weights='imagenet', classes=n_classes, activation=activation)

# compile keras model with defined optimozer, loss and metrics
model1.compile(optim, total_loss, metrics=metrics)

In [ ]:
# print(model1.summary())

In [ ]:
history1=model1.fit(X_train1, 
          y_train_cat,
          batch_size=8, 
          epochs=20,
          verbose=1,
          validation_data=(X_test1, y_test_cat))


In [ ]:
model1.save('UNet_res50_backbone_20epochs.hdf5')

In [ ]:
model = model1.compile(optim, total_loss, metrics=metrics)
# Evaluate the model on the test set
evaluation = model1.evaluate(x=X_test1, y= y_test_cat, batch_size=8)

# Extract the accuracy from the evaluation
accuracy = evaluation[1] * 100  # Assuming accuracy is the second metric

# Print the accuracy
print(f"Test set accuracy: {accuracy:.2f}%")

In [ ]:
loss = history1.history['loss']
val_loss = history1.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history1.history['iou_score']
val_acc = history1.history['val_iou_score']

plt.plot(epochs, acc, 'y', label='Training IOU')
plt.plot(epochs, val_acc, 'r', label='Validation IOU')
plt.title('Training and validation IOU')
plt.xlabel('Epochs')
plt.ylabel('IOU')
plt.legend()
plt.show()

In [ ]:
from keras.models import load_model
model1 = load_model('UNet_res50_backbone_20epochs.hdf5', compile=False)
#IOU
y_pred1=model1.predict(X_test1)
y_pred1_argmax=np.argmax(y_pred1, axis=3)

In [ ]:
#Using built in keras function
#from keras.metrics import MeanIoU
n_classes = 6
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(y_test[:,:,:,0], y_pred1_argmax)
print("Mean IoU =", IOU_keras.result().numpy())

In [ ]:
#To calculate I0U for each class...
values = np.array(IOU_keras.get_weights()).reshape(n_classes, n_classes)
print(values)
class1_IoU = values[0,0]/(values[0,0] + values[0,1] + values[0,2] + values[0,3] + values[1,0]+ values[2,0]+ values[3,0])
class2_IoU = values[1,1]/(values[1,1] + values[1,0] + values[1,2] + values[1,3] + values[0,1]+ values[2,1]+ values[3,1])
class3_IoU = values[2,2]/(values[2,2] + values[2,0] + values[2,1] + values[2,3] + values[0,2]+ values[1,2]+ values[3,2])
class4_IoU = values[3,3]/(values[3,3] + values[3,0] + values[3,1] + values[3,2] + values[0,3]+ values[1,3]+ values[2,3])
class5_IoU = values[4,4]/(values[4,4] + values[4,0] + values[4,1] + values[4,2] + values[0,4]+ values[1,4]+ values[2,4])
class6_IoU = values[5,5]/(values[5,5] + values[5,0] + values[5,1] + values[5,2] + values[0,5]+ values[1,5]+ values[2,5])

print("IoU for class1: Building is: ", class1_IoU)
print("IoU for class2: Impervious Surface is: ", class2_IoU)
print("IoU for class3: Trees is: ", class3_IoU)
print("IoU for class4: Low Vegetation is: ", class4_IoU)
print("IoU for class5: Car is: ", class5_IoU)
print("IoU for class6: Clutter/Background is: ", class6_IoU)

In [ ]:
import random
test_img_number = random.randint(0, len(X_test1))
test_img = X_test1[test_img_number]
ground_truth=y_test[test_img_number]
test_img_input=np.expand_dims(test_img, 0)

test_img_input1 = preprocess_input1(test_img_input)

test_pred1 = model1.predict(test_img_input1)
test_prediction1 = np.argmax(test_pred1, axis=3)[0,:,:]


plt.figure(figsize=(12, 8))
plt.subplot(231)
plt.title('Testing Image')
plt.imshow(test_img[:,:,0], cmap='gray')
plt.subplot(232)
plt.title('Testing Label')
plt.imshow(ground_truth[:,:,0], cmap='jet')
plt.subplot(233)
plt.title('Prediction on test image')
plt.imshow(test_prediction1, cmap='jet')
plt.show()
print(test_img_number)